In [ ]:
%matplotlib widget

In [ ]:
# пересохраним файл, чтобы были только нужные данные, чтобы потом каждый раз грузить их чуть быстрей.
import numpy as np
import pandas as pd

# Загружаем исходный npz файл
file_path = "../npz/BTCUSD_1T.npz"
data = np.load(file_path, allow_pickle=True)
raw_data = data['data']
# Преобразуем данные в DataFrame
df = pd.DataFrame(raw_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
# Оставляем только timestamp и close
filtered_df = df[['timestamp', 'close']]
# Сохраняем новый npz файл с отфильтрованными данными
np.savez_compressed('../npz/BTCUSD_1T_close_only.npz', data=filtered_df.to_numpy())

In [ ]:
# загрузим данные
import numpy as np
import pandas as pd

file_path = "../npz/BTCUSD_1T_close_only.npz"
data = np.load(file_path, allow_pickle=True)
df = pd.DataFrame(data['data'], columns=['timestamp', 'close'])
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)

# При необходимости можно ограничить число точек:
#df = df.iloc[:50* 1000]


In [ ]:
from scipy.signal import savgol_filter
full_len = 70
window = 31
dfc = df.copy().iloc[:full_len]

# делаю савгол для всех данных
full = savgol_filter(dfc['close'].values, window_length=window, polyorder=3, deriv=0, mode='nearest')
# делаю савгол для урезанных данных. это симуляция того, что мы находимся назад во времени и не видим то, что будет в будущем
truncated = savgol_filter(dfc.iloc[:-window]['close'].values, window_length=window, polyorder=3, deriv=0, mode='nearest')
# подрежем теперь данные, чтобы у всех массивов была длина как у truncated, чтобы отрисовать их все на одном графике
full = full[:-window]
df_fixed = dfc[:-window]

import matplotlib.pyplot as plt
fig, (ax, ax2) = plt.subplots(
    2,  # Два графика (2 строки)
    1,  # Один столбец
    figsize=(9, 7),  # Общий размер
    gridspec_kw={'height_ratios': [2, 2]}
)
ax.plot(df_fixed.index, df_fixed['close'], label='Original', color='blue', marker='o', markersize=3)
ax.plot(df_fixed.index, full, label='Full', color='black', marker='o', markersize=3)
ax.plot(df_fixed.index, truncated, label='Truncated', color='red')
ax.legend()
ax2.plot(df_fixed.index, truncated - full, label='difference', marker='o', markersize=3)


In [ ]:
from scipy.signal import savgol_filter
from numpy.polynomial.polynomial import Polynomial

full_len = 90
window = 31
dfc = df.copy().iloc[:full_len]

# делаю савгол для всех данных
full = savgol_filter(dfc['close'].values, window_length=window, polyorder=3, deriv=0)
# делаю савгол для урезанных данных. это симуляция того, что мы находимся назад во времени и не видим то, что будет в будущем

def backward_sma_extrapolate(series, n_extrapolate, n_fit):
    """Экстраполяция на основе обратного EMA (разворот данных перед сглаживанием)"""
    x = np.arange(len(series))
    y = series.values

    # Берём последние n_fit точек и разворачиваем их
    y_reversed = np.flip(y[-n_fit:])

    # Делаем обычный EMA на перевёрнутых данных
    y_smooth = pd.Series(y_reversed).rolling(window=n_fit, min_periods=1).mean().values
    #y_smooth = pd.Series(y_reversed).ewm(alpha=alpha, adjust=False).mean().values

    # Разворачиваем обратно
    y_smooth = np.flip(y_smooth)

    # Теперь считаем наклон по сглаженным данным
    slope = (y_smooth[-1] - y_smooth[0]) / (n_fit - 1)

    # Экстраполяция
    y_extra = y_smooth[-1] + slope * np.arange(1, n_extrapolate + 1)
    
    return np.concatenate([y, y_extra])

extrapolated_sma = backward_sma_extrapolate(dfc['close'].iloc[:-window], n_extrapolate=window, n_fit=window)
truncated = savgol_filter(extrapolated_sma, window_length=window, polyorder=3, deriv=0)
# подрежем теперь данные, чтобы у всех массивов была длина как у truncated, чтобы отрисовать их все на одном графике
full = full[:-window]
df_fixed = dfc[:-window]
truncated = truncated[:-window]

import matplotlib.pyplot as plt
fig, (ax, ax2) = plt.subplots(
    2,  # Два графика (2 строки)
    1,  # Один столбец
    figsize=(9, 7),  # Общий размер
    gridspec_kw={'height_ratios': [2, 2]}
)
ax.plot(df_fixed.index, df_fixed['close'], label='Original', color='blue', marker='o', markersize=3)
ax.plot(df_fixed.index, full, label='Full', color='black', marker='o', markersize=3)
ax.plot(df_fixed.index, truncated, label='Truncated', color='red')
ax.legend()
ax2.plot(df_fixed.index, truncated - full, label='difference', marker='o', markersize=3)


In [ ]:
dfc = df.copy()
dfc = dfc[:150]
def backward_ema_extrapolate(series, n_extrapolate=10, n_fit=20, alpha=0.3):
    """Экстраполяция на основе обратного EMA (разворот данных перед сглаживанием)"""
    x = np.arange(len(series))
    y = series.values

    # Берём последние n_fit точек и разворачиваем их
    y_reversed = np.flip(y[-n_fit:])

    # Делаем обычный EMA на перевёрнутых данных
    #y_smooth = pd.Series(y_reversed).rolling(window=n_fit, min_periods=1).mean().values
    y_smooth = pd.Series(y_reversed).ewm(alpha=alpha, adjust=False).mean().values

    # Разворачиваем обратно
    y_smooth = np.flip(y_smooth)

    # Теперь считаем наклон по сглаженным данным
    slope = (y_smooth[-1] - y_smooth[0]) / (n_fit - 1)

    # Экстраполяция
    y_extra = y_smooth[-1] + slope * np.arange(1, n_extrapolate + 1)
    
    return np.concatenate([y, y_extra])

def backward_sma_extrapolate(series, n_extrapolate, n_fit):
    """Экстраполяция на основе обратного EMA (разворот данных перед сглаживанием)"""
    x = np.arange(len(series))
    y = series.values

    # Берём последние n_fit точек и разворачиваем их
    y_reversed = np.flip(y[-n_fit:])

    # Делаем обычный EMA на перевёрнутых данных
    y_smooth = pd.Series(y_reversed).rolling(window=n_fit, min_periods=1).mean().values
    #y_smooth = pd.Series(y_reversed).ewm(alpha=alpha, adjust=False).mean().values

    # Разворачиваем обратно
    y_smooth = np.flip(y_smooth)

    # Теперь считаем наклон по сглаженным данным
    slope = (y_smooth[-1] - y_smooth[0]) / (n_fit - 1)

    # Экстраполяция
    y_extra = y_smooth[-1] + slope * np.arange(1, n_extrapolate + 1)
    
    return np.concatenate([y, y_extra])

extrapolated_ema = backward_ema_extrapolate(dfc['close'].iloc[:-window], n_extrapolate=window, n_fit=window, alpha=0.2)
extrapolated_sma = backward_sma_extrapolate(dfc['close'].iloc[:-window], n_extrapolate=window, n_fit=window)

fig, ax = plt.subplots(figsize=(8,4))
ax.plot(dfc.index, dfc.shift(window).values, label='shifted', color='gray', marker='o', markersize=4)
ax.plot(dfc.index, dfc.values, label='dfc', color='black', marker='o', markersize=4)
ax.plot(dfc.index, extrapolated_ema, label='extrapolated_ema', color='red', marker='o', markersize=2)
ax.plot(dfc.index, extrapolated_sma, label='extrapolated_sma', color='pink', marker='o', markersize=2)
plt.legend()
plt.tight_layout()


In [ ]:
full1 = savgol_filter(dfc['close'].values, window_length=window, polyorder=3, deriv=1)
truncated1 = savgol_filter(dfc.iloc[:-window]['close'].values, window_length=window, polyorder=3, deriv=1)

fig, ax = plt.subplots()
ax.plot(dfc.index[:-window], full1[:-window], label='full1', color='black')
ax.plot(dfc.index[:-window], truncated1, label='truncated1', color='red')
fig, ax = plt.subplots()
ax.plot(dfc.index[:-window], truncated1 - full1[:-window], label='difference1')

In [ ]:
from scipy.signal import savgol_filter
from numpy.polynomial.polynomial import Polynomial

full_len = 90
window = 31
dfc = df.copy().iloc[:full_len]

# делаю савгол для всех данных
full = savgol_filter(dfc['close'].values, window_length=window, polyorder=3, deriv=0)
# делаю савгол для урезанных данных. это симуляция того, что мы находимся назад во времени и не видим то, что будет в будущем

def backward_sma_extrapolate(series, n_extrapolate, n_fit):
    """Экстраполяция на основе обратного EMA (разворот данных перед сглаживанием)"""
    x = np.arange(len(series))
    y = series.values

    # Берём последние n_fit точек и разворачиваем их
    y_reversed = np.flip(y[-n_fit:])

    # Делаем обычный EMA на перевёрнутых данных
    y_smooth = pd.Series(y_reversed).rolling(window=n_fit, min_periods=1).mean().values
    #y_smooth = pd.Series(y_reversed).ewm(alpha=alpha, adjust=False).mean().values

    # Разворачиваем обратно
    y_smooth = np.flip(y_smooth)

    # Теперь считаем наклон по сглаженным данным
    slope = (y_smooth[-1] - y_smooth[0]) / (n_fit - 1)

    # Экстраполяция
    y_extra = y_smooth[-1] + slope * np.arange(1, n_extrapolate + 1)
    
    return np.concatenate([y, y_extra])

extrapolated_sma = backward_sma_extrapolate(dfc['close'].iloc[:-window], n_extrapolate=window, n_fit=window)
truncated = savgol_filter(extrapolated_sma, window_length=window, polyorder=3, deriv=0)
# подрежем теперь данные, чтобы у всех массивов была длина как у truncated, чтобы отрисовать их все на одном графике
full = full[:-window]
df_fixed = dfc[:-window]
truncated = truncated[:-window]

import matplotlib.pyplot as plt
fig, (ax, ax2) = plt.subplots(
    2,  # Два графика (2 строки)
    1,  # Один столбец
    figsize=(9, 7),  # Общий размер
    gridspec_kw={'height_ratios': [2, 2]}
)
ax.plot(df_fixed.index, df_fixed['close'], label='Original', color='blue', marker='o', markersize=3)
ax.plot(df_fixed.index, full, label='Full', color='black', marker='o', markersize=3)
#ax.plot(df_fixed.index, truncated, label='Truncated', color='red')
from statsmodels.nonparametric.smoothers_lowess import lowess
ax.plot(df_fixed.index, lowess(df_fixed['close'], df_fixed.index, frac=0.3)[:, 1], label='lowess', color='green')
from scipy.ndimage import gaussian_filter1d
ax.plot(df_fixed.index, gaussian_filter1d(df_fixed['close'].astype(float).values, sigma=3), label='gaussian_filter1d', color='red')

ax.legend()
ax2.plot(df_fixed.index, truncated - full, label='difference', marker='o', markersize=3)
